In [ ]:
import pretty_midi
import numpy as np
import muspy
import os
import csv

# Run parse_data
%run ~/violin-renderer/src/data/parse_data.ipynb

# Install soundfont for muspy
muspy.download_musescore_soundfont()

In [ ]:
# output a CSV file containing the given expressive start and end timings
# @param:
    # start_timings: an array of expressive start times
    # end_timings: an array of expressive end times
    # output_path: path to a directory to store file
    # file_name: name of CSV file
def store_CSV(start_timings, end_timings, output_path, file_name):
    if len(start_timings) != len(end_timings):
        raise ValueError("Arrays must have the same length")
    else:
        rows = zip(start_timings, end_timings)
        with open(output_path + file_name + '.csv', mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['start', 'end'])  # Write header
            writer.writerows(rows)

In [ ]:
# output a wav file in folder that converts the expressive timings
# @param:
    # input_path: path to the input file
    # tempo_path: path to the tempo file
    # output_path: path to a directory to store file
    # name: name of the .wav file
def synthesize_source_input(input_path, tempo_path, output_path, name):
    MIDI = pretty_midi.PrettyMIDI()

    # Create an Instrument instance for a violin instrument
    violin_program = pretty_midi.instrument_name_to_program('Violin')
    violin = pretty_midi.Instrument(program=violin_program)

    # Create notes
    input_info = parse_source_input_file(input_path)
    tempo = parse_tempo_file(tempo_path)    

    # Add notes to instrument
    for notes in input_info:
        # We convert from tick to actual timings in seconds
        start = (notes[0] / 24) / tempo * 60
        end = (notes[1] / 24) / tempo * 60
        note = pretty_midi.Note(
            velocity=64, pitch=int(notes[2]), start=start, end=end)
        violin.notes.append(note)

    # Add the instrument to the PrettyMIDI object
    MIDI.instruments.append(violin)

    # Write to a MIDI file
    MIDI.write('violin.mid')

    # Convert to WAV using fluidsynth
    soundfont_path = os.path.expanduser("~") + '/.muspy/musescore-general/MuseScore_General.sf3'
    os.system(f'fluidsynth -ni {soundfont_path} {"violin.mid"} -F {output_path + name + ".wav"} -r 44100')

    # Remove the MIDI file
    os.remove('violin.mid')

In [ ]:
# output a wav file in src folder that converts the expressive timings
# @param:
    # notes_path: path to the original input file -> to get pitch and velocity
    # timings_path: path to the generative output file containing the expressive start and end timings
    # output_path: path to a directory to store file
    # name: name of the .wav file
def synthesize_ground_truth(notes_path, timings_path, output_path, name):
    MIDI = pretty_midi.PrettyMIDI()

    # Create an Instrument instance for a violin instrument
    violin_program = pretty_midi.instrument_name_to_program('Violin')
    violin = pretty_midi.Instrument(program=violin_program)

    # Create notes
    notes_info = parse_source_input_file(notes_path)
    timings_info = parse_ground_truth_file(timings_path)

    for notes, times in zip(notes_info, timings_info):
        note = pretty_midi.Note(
            velocity=64, pitch=int(notes[2]), start=times[0], end=times[1])
        violin.notes.append(note)

    # Add the instrument to the PrettyMIDI object
    MIDI.instruments.append(violin)

    # Write to a MIDI file
    MIDI.write('violin.mid')

    # Convert to WAV using fluidsynth
    soundfont_path = os.path.expanduser("~") + '/.muspy/musescore-general/MuseScore_General.sf3'
    os.system(f'fluidsynth -ni {soundfont_path} {"violin.mid"} -F {output_path + name + ".wav"} -r 44100')

    # Remove the MIDI file
    os.remove('violin.mid')

In [ ]:
# output a wav file in src folder that converts the expressive timings
# @param:
    # notes_path: path to the original input file -> to get pitch and velocity
    # generated_timings: start and end times in the format [(start, end),...]
    # output_path: path to a directory to store file
    # name: name of the .wav file
def synthesize_generated_output(notes_path, generated_timings, output_path, name):
    MIDI = pretty_midi.PrettyMIDI()
    
    # Create an Instrument instance for a violin instrument
    violin_program = pretty_midi.instrument_name_to_program('Violin')
    violin = pretty_midi.Instrument(program=violin_program)
    
    # Create notes
    note_values = parse_source_input_file(notes_path)

    for notes, times in zip(note_values, generated_timings):
        # If start < end
        if times[0] < times[1]:
            note = pretty_midi.Note(
                velocity=64, pitch=int(notes[2]), start=times[0], end=times[1])
            violin.notes.append(note)

    # Add the instrument to the PrettyMIDI object
    MIDI.instruments.append(violin)

    # Write to a MIDI file
    MIDI.write('violin.mid')

    # Convert to WAV using fluidsynth
    soundfont_path = os.path.expanduser("~") + '/.muspy/musescore-general/MuseScore_General.sf3'
    os.system(f'fluidsynth -ni {soundfont_path} {"violin.mid"} -F {output_path + name + ".wav"} -r 44100')

    # Remove the MIDI file
    os.remove('violin.mid')